# Prática Independente - Regressão IV.

## Dados abertos do Airbnb no Rio de Janeiro:

#### Desde 2008, os hóspedes e anfitriões têm usado o Airbnb para expandir as possibilidades de viagem e apresentar uma forma mais única e personalizada de experimentar o mundo. Este conjunto de dados descreve a atividade de listagem e as métricas em Rio de Janeiro, RJ para 2020.

### Conteúdo:

#### O arquivo `'AB_RJ_2020.csv'` inclui todas as informações necessárias para descobrir mais sobre hosts, disponibilidade geográfica, métricas necessárias para fazer previsões e tirar conclusões.

#### Este conjunto de dados [públicos](http://insideairbnb.com/) faz parte do [Airbnb](https://www.airbnb.com.br/).

<img src="RegMetroRJ.png" width="1532" height="1037" align="center"/>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import statsmodels.api as sm
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Tamanho padrões das figuras e a fonte de seus textos neste notebook
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 14

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def EDA (df):
    eda_df = {}
    eda_df['Amount_NaN'] = df.isnull().sum()
    eda_df['%_NaN'] = df.isnull().mean().round(2)
    eda_df['DType'] = df.dtypes
    eda_df['Amount_Data'] = df.count()
    
    # Outro ponto para ser verificado, porque para criar a coluna com a quantidade de valores unicos por coluna
    # Não utilizei a função df.unique() 
    colunas = sorted(df.columns.tolist(), key=str.lower, reverse=False)
    eda_df['Amount_Unique'] = filtered_result = list(map(lambda x: len(df[x].unique().tolist()), colunas))
    
    eda_df['Mean'] = np.round(df.mean(), 2)
    eda_df['Median'] = np.round(df.median(), 2)
    
    eda_df['Max'] = df.max()
    eda_df['Min'] = df.min()
    eda_df['STD'] = np.round(df.std(), 2)
    
    return pd.DataFrame(eda_df)

#### Exercício 1 - Realize uma análise exploratória nos dados abertos do AirBNB para a cidade do Rio de Janeiro.

In [ ]:
dados = pd.read_csv('AB_RJ_2020.csv', parse_dates= True, index_col= 'id')
dados.head()

,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
id,,,,,,,,,,,,,,
17878,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",68997,Matthias,"Rio de Janeiro, Brazil",-22.96592,-43.17896,Entire home/apt,$500.00,5,259,2020-12-26,2.03,1,0
25026,Beautiful Modern Decorated Studio in Copa,3746246,Ghizlane,"Rio de Janeiro, Brazil",-22.97712,-43.19045,Entire home/apt,$160.00,7,238,2020-02-15,1.85,11,335
35636,Cosy flat close to Ipanema beach,153232,Patricia,"Rio de Janeiro, Brazil",-22.98816,-43.19359,Entire home/apt,$259.00,2,181,2020-03-15,2.07,1,267
35764,COPACABANA SEA BREEZE - RIO - 20 X Superhost,153691,Patricia Miranda & Paulo,"Rio de Janeiro, Brazil",-22.98127,-43.19046,Entire home/apt,$157.00,3,346,2020-12-20,2.78,1,89
41198,"Modern 2bed,Top end of Copacabana",178975,Nicky,NaN,-22.97962,-43.19230,Entire home/apt,"$1,035.00",3,18,2016-02-09,0.19,2,365


In [ ]:
print(f'Instâncias: {dados.shape[0]}')
print(f'Variáveis: {dados.shape[1]}')

Instâncias: 25784
Variáveis: 14


In [ ]:
EDA(dados)

,Amount_NaN,%_NaN,DType,Amount_Data,Amount_Unique,Mean,Median,Max,Min,STD
availability_365,0,0.00,int64,25784,366,215.30,247.00,365,0,1.408900e+02
calculated_host_listings_count,0,0.00,int64,25784,48,6.58,1.00,200,1,2.000000e+01
host_id,0,0.00,int64,25784,16946,96308131.49,58502919.50,381289871,3607,1.041834e+08
host_name,8,0.00,object,25776,5076,NaN,NaN,NaN,NaN,NaN
last_review,9932,0.39,object,15852,1447,NaN,NaN,NaN,NaN,NaN
latitude,0,0.00,float64,25784,9782,-22.97,-22.97,-22.7498,-23.0729,3.000000e-02
longitude,0,0.00,float64,25784,12093,-43.25,-43.20,-43.1049,-43.7048,1.000000e-01
minimum_nights,0,0.00,int64,25784,67,4.77,2.00,1000,1,1.904000e+01
name,29,0.00,object,25755,25086,NaN,NaN,NaN,NaN,NaN
neighbourhood,11675,0.45,object,14109,270,NaN,NaN,NaN,NaN,NaN


1) Temos 25.784 instâncias no dataset e 14 variáveis a priori;

2) Temos muitos dados nulos nas variáveis last_review, neighbourhood e reviews_per_month

3) Precisamos tratar os dados da coluna price para tranformá-los em tipo Inteiro para utilizarmos no modelo de regressão com o tipo correto

4) Temos uma media de 215 em que os hosts estão disponíveis, por ano, com uma mediana de 247 e moda de 365. A maioria está disponivel o ano todo, porém possuímos outliers com o número de 0 disponíveis por ano que podem estar puxando a media pra baixo.

5) Olhando mediana e media podemos indicar que a maioria dos dados de minímo de noite (para a paga do valor indicado) está entre 2 e 5).

In [ ]:
dados['host_name'].value_counts()

Daniel               318
Maria                270
Marcelo              243
Ricardo              212
Carlos               196
                    ... 
Aldaíza  Aldado        1
Elfi                   1
Paulo Henrique         1
Vanize                 1
Luciano E Gláucio      1
Name: host_name, Length: 5075, dtype: int64

Ao contar o número de hosts por pessoas vemos muitas que se repetem. Isso pode se dar porque só o primeiro nome está contido e assim são pessoas diferentes com nomes iguais, ou pode haver pessoas com maior número de locais disponiveis na plataforma Airbnd e outras que colocam a vaga de forma mais eventual.

#### Exercício 2 -Faça a limpeza dos dados.

In [ ]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25784 entries, 17878 to 47221763
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   name                            25755 non-null  object 
 1   host_id                         25784 non-null  int64  
 2   host_name                       25776 non-null  object 
 3   neighbourhood                   14109 non-null  object 
 4   latitude                        25784 non-null  float64
 5   longitude                       25784 non-null  float64
 6   room_type                       25784 non-null  object 
 7   price                           25784 non-null  object 
 8   minimum_nights                  25784 non-null  int64  
 9   number_of_reviews               25784 non-null  int64  
 10  last_review                     15852 non-null  object 
 11  reviews_per_month               15852 non-null  float64
 12  calculated_host_listings_

Vamos retirar colunas que não serão utilizadas no modelo e passar os nomes daquelas que possuem variáveis categóricas para variáveis numéricas.

Ao explorar a coluna neighbourhood encontramos hosts do Bairro Rio de Janeiro escrito de duas formas diferentes: 'río (ascento agudo no i) de janeiro e rio (sem acento agudo no i) de janeiro. Vamos padronizar os dados retirando o acento agudo desses i's. Vamos aproveitar e passar todos os seus nomes para minúsculos e fazer o mesmo para a coluna roo_type. Assim evitamos que os mesmos tipos de quartos ou mesmos bairros sejam codificados como diferentes por estarem escritos de forma diferente.

In [ ]:
#Retirando colunas que não serão utilizadas
dados.drop(labels= ['host_name', 'host_id', 'name', 'last_review'], axis= 1, inplace= True)

#Passando valores de colunas categóricas para letras minúsculas
colunas_minusculas = ['neighbourhood', 'room_type']
for coluna in colunas_minusculas:
  dados[coluna].str.lower()

In [ ]:
#Padronizando nome do bairro rio de janeiro
dados['neighbourhood']= dados['neighbourhood'].str.replace('í', 'i', regex= True)

Vamos transformar a coluna price de object para int

In [ ]:
#Modificando coluna de preço de object para int
dados['price']= dados['price'].str.replace('$', '', regex= True)
dados['price']= dados['price'].str.replace(',', '', regex= True)
dados['price']= dados['price'].str.replace('.', '', regex= True)
dados['price'] = pd.to_numeric(dados['price'])

dados['price'].head()

id
17878     50000
25026     16000
35636     25900
35764     15700
41198    103500
Name: price, dtype: int64

Vamos preencher os valores faltantes numericos usando o algoritmo [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) que distribui os dados em pontos n-dimensionais (sendo n o total de variáveis independentes) e prediz um valor para o dado faltante baseado na sua proximidade nesse espaço com pontos que não possuem dados faltantes. Primeiro, porém, temos de codificar as variáveis categóricas com dados numéricos em nosso dataset.

In [ ]:
#Instanciado objeto OrdinalEncoder
encoder = OrdinalEncoder()

def encode(data):
    '''função que codifica valores não nulos e substitui no dataset original'''
    #retira os valores nulos dataset
    nao_nulos = np.array(data.dropna())
    #reshape dos dados para codificação
    impute_reshape = nao_nulos.reshape(-1,1)
    #codifica os dados
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Atribui os valores codificados de volta a valores não nulos
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

In [ ]:
#Codificando colunas categóricas de nossos dados
colunas_categoricas= ['neighbourhood', 'room_type']

for coluna in colunas_categoricas:
    encode(dados[coluna])

dados.sample(4)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
id,,,,,,,,,,
38381931,54,-22.97481,-43.18867,0.0,25700,2,15,1.18,3,134
46745536,NaN,-22.96251,-43.22196,0.0,27000,2,0,NaN,4,243
9164215,181,-23.01294,-43.29212,0.0,500000,7,0,NaN,44,179
40610868,94,-22.98368,-43.19545,2.0,80000,5,1,0.09,1,82


Agora que temos nossa coluna de bairros com valores numéricos representando cada bairro, podemos fazer o preenchimento dos valores faltantes utilizando o KNNImputer

In [ ]:
#Instanciando objeto KNNImputer com 7 números vizinhos de parametro, pesos iguais para todos os vizinhos dentro desses 7
#E distância euclidiana para calcular as distâncias entre os pontos
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

#Treinando modelo
imputer.fit(dados)

#Transformando o dataset original com os valores nulos implementados
dados_transf = imputer.transform(dados)

#Retornando os dados para um dataframe pandas com as categorias em formato numérico
dados_transf = pd.DataFrame(imputer.fit_transform(dados_transf),columns = dados.columns)
dados_transf.head()

,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,181.0,-22.96592,-43.17896,0.0,50000.0,5.0,259.0,2.03,1.0,0.0
1,181.0,-22.97712,-43.19045,0.0,16000.0,7.0,238.0,1.85,11.0,335.0
2,181.0,-22.98816,-43.19359,0.0,25900.0,2.0,181.0,2.07,1.0,267.0
3,181.0,-22.98127,-43.19046,0.0,15700.0,3.0,346.0,2.78,1.0,89.0
4,168.6,-22.97962,-43.19230,0.0,103500.0,3.0,18.0,0.19,2.0,365.0


In [ ]:
dados_transf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25784 entries, 0 to 25783
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood                   25784 non-null  float64
 1   latitude                        25784 non-null  float64
 2   longitude                       25784 non-null  float64
 3   room_type                       25784 non-null  float64
 4   price                           25784 non-null  float64
 5   minimum_nights                  25784 non-null  float64
 6   number_of_reviews               25784 non-null  float64
 7   reviews_per_month               25784 non-null  float64
 8   calculated_host_listings_count  25784 non-null  float64
 9   availability_365                25784 non-null  float64
dtypes: float64(10)
memory usage: 2.0 MB


#### Exercício 3 - Analise os dados e realize a anonimização dos mesmos. 

Já o fizemos ao retirar as colunas que dizem respeito ao nomes dos hosts e nome dos donos dos hosts no exercício 2.

#### Exercício 4 - Crie um modelo de regressão para os preços dos apartamentos.